In [13]:
import gzip
import json
import copy
import os
import pandas as pd
from collections import defaultdict

from PIL import ImageFont, ImageDraw
from PIL import Image as PImage

In [17]:
df_categories = pd.DataFrame.from_dict([{'id': 1, 'name': 'person', 'supercategory': 'person'},
 {'id': 2, 'name': 'bicycle', 'supercategory': 'vehicle'},
 {'id': 3, 'name': 'car', 'supercategory': 'vehicle'},
 {'id': 4, 'name': 'motorcycle', 'supercategory': 'vehicle'},
 {'id': 5, 'name': 'airplane', 'supercategory': 'vehicle'},
 {'id': 6, 'name': 'bus', 'supercategory': 'vehicle'},
 {'id': 7, 'name': 'train', 'supercategory': 'vehicle'},
 {'id': 8, 'name': 'truck', 'supercategory': 'vehicle'},
 {'id': 9, 'name': 'boat', 'supercategory': 'vehicle'},
 {'id': 10, 'name': 'traffic light', 'supercategory': 'outdoor'},
 {'id': 11, 'name': 'fire hydrant', 'supercategory': 'outdoor'},
 {'id': 13, 'name': 'stop sign', 'supercategory': 'outdoor'},
 {'id': 14, 'name': 'parking meter', 'supercategory': 'outdoor'},
 {'id': 15, 'name': 'bench', 'supercategory': 'outdoor'},
 {'id': 16, 'name': 'bird', 'supercategory': 'animal'},
 {'id': 17, 'name': 'cat', 'supercategory': 'animal'},
 {'id': 18, 'name': 'dog', 'supercategory': 'animal'},
 {'id': 19, 'name': 'horse', 'supercategory': 'animal'},
 {'id': 20, 'name': 'sheep', 'supercategory': 'animal'},
 {'id': 21, 'name': 'cow', 'supercategory': 'animal'},
 {'id': 22, 'name': 'elephant', 'supercategory': 'animal'},
 {'id': 23, 'name': 'bear', 'supercategory': 'animal'},
 {'id': 24, 'name': 'zebra', 'supercategory': 'animal'},
 {'id': 25, 'name': 'giraffe', 'supercategory': 'animal'},
 {'id': 27, 'name': 'backpack', 'supercategory': 'accessory'},
 {'id': 28, 'name': 'umbrella', 'supercategory': 'accessory'},
 {'id': 31, 'name': 'handbag', 'supercategory': 'accessory'},
 {'id': 32, 'name': 'tie', 'supercategory': 'accessory'},
 {'id': 33, 'name': 'suitcase', 'supercategory': 'accessory'},
 {'id': 34, 'name': 'frisbee', 'supercategory': 'sports'},
 {'id': 35, 'name': 'skis', 'supercategory': 'sports'},
 {'id': 36, 'name': 'snowboard', 'supercategory': 'sports'},
 {'id': 37, 'name': 'sports ball', 'supercategory': 'sports'},
 {'id': 38, 'name': 'kite', 'supercategory': 'sports'},
 {'id': 39, 'name': 'baseball bat', 'supercategory': 'sports'},
 {'id': 40, 'name': 'baseball glove', 'supercategory': 'sports'},
 {'id': 41, 'name': 'skateboard', 'supercategory': 'sports'},
 {'id': 42, 'name': 'surfboard', 'supercategory': 'sports'},
 {'id': 43, 'name': 'tennis racket', 'supercategory': 'sports'},
 {'id': 44, 'name': 'bottle', 'supercategory': 'kitchen'},
 {'id': 46, 'name': 'wine glass', 'supercategory': 'kitchen'},
 {'id': 47, 'name': 'cup', 'supercategory': 'kitchen'},
 {'id': 48, 'name': 'fork', 'supercategory': 'kitchen'},
 {'id': 49, 'name': 'knife', 'supercategory': 'kitchen'},
 {'id': 50, 'name': 'spoon', 'supercategory': 'kitchen'},
 {'id': 51, 'name': 'bowl', 'supercategory': 'kitchen'},
 {'id': 52, 'name': 'banana', 'supercategory': 'food'},
 {'id': 53, 'name': 'apple', 'supercategory': 'food'},
 {'id': 54, 'name': 'sandwich', 'supercategory': 'food'},
 {'id': 55, 'name': 'orange', 'supercategory': 'food'},
 {'id': 56, 'name': 'broccoli', 'supercategory': 'food'},
 {'id': 57, 'name': 'carrot', 'supercategory': 'food'},
 {'id': 58, 'name': 'hot dog', 'supercategory': 'food'},
 {'id': 59, 'name': 'pizza', 'supercategory': 'food'},
 {'id': 60, 'name': 'donut', 'supercategory': 'food'},
 {'id': 61, 'name': 'cake', 'supercategory': 'food'},
 {'id': 62, 'name': 'chair', 'supercategory': 'furniture'},
 {'id': 63, 'name': 'couch', 'supercategory': 'furniture'},
 {'id': 64, 'name': 'potted plant', 'supercategory': 'furniture'},
 {'id': 65, 'name': 'bed', 'supercategory': 'furniture'},
 {'id': 67, 'name': 'dining table', 'supercategory': 'furniture'},
 {'id': 70, 'name': 'toilet', 'supercategory': 'furniture'},
 {'id': 72, 'name': 'tv', 'supercategory': 'electronic'},
 {'id': 73, 'name': 'laptop', 'supercategory': 'electronic'},
 {'id': 74, 'name': 'mouse', 'supercategory': 'electronic'},
 {'id': 75, 'name': 'remote', 'supercategory': 'electronic'},
 {'id': 76, 'name': 'keyboard', 'supercategory': 'electronic'},
 {'id': 77, 'name': 'cell phone', 'supercategory': 'electronic'},
 {'id': 78, 'name': 'microwave', 'supercategory': 'appliance'},
 {'id': 79, 'name': 'oven', 'supercategory': 'appliance'},
 {'id': 80, 'name': 'toaster', 'supercategory': 'appliance'},
 {'id': 81, 'name': 'sink', 'supercategory': 'appliance'},
 {'id': 82, 'name': 'refrigerator', 'supercategory': 'appliance'},
 {'id': 84, 'name': 'book', 'supercategory': 'indoor'},
 {'id': 85, 'name': 'clock', 'supercategory': 'indoor'},
 {'id': 86, 'name': 'vase', 'supercategory': 'indoor'},
 {'id': 87, 'name': 'scissors', 'supercategory': 'indoor'},
 {'id': 88, 'name': 'teddy bear', 'supercategory': 'indoor'},
 {'id': 89, 'name': 'hair drier', 'supercategory': 'indoor'},
 {'id': 90, 'name': 'toothbrush', 'supercategory': 'indoor'}])

dict_keys(['status', 'qas', 'questioner_id', 'timestamp', 'image', 'object_id', 'objects', 'id'])

In [ ]:
%%time
file = '../../data/guesswhat.valid.jsonl.gz'
games = defaultdict(list)
with gzip.open(file) as f:
    for i, line in enumerate(f):
        categories = defaultdict(list)
        line = line.decode("utf-8")
        jsonline = json.loads(line.strip('\n'))
        status = jsonline['status']
        game_id = jsonline['id']
        for v in jsonline['objects']:
            category = v['category']
            categories['category'].append(category)
            category_id = v['category_id']
            categories['category_id'].append(category_id)
            supercategory = df_categories[df_categories.id == category_id].name.iloc[0]
            categories['supercategory'].append(supercategory)
            if v['id'] == jsonline['object_id']:
                target_category = v['category']
                target_category_id = v['category_id']
                target_supercategory = df_categories[df_categories.id == target_category_id].name.iloc[0]
                target_supercategory_id = df_categories[df_categories.id == target_category_id].name.index.values[0]
        categories = str(dict(categories))
        for v in jsonline['qas']:
            question = v['question']
            answer = v['answer']
            qas_id = v['id']
            games['game_id'].append(game_id)
            games['question'].append(question)
            games['answer'].append(answer)
            games['question_id'].append(qas_id)
            games['target_category'].append(target_category)
            games['target_category_id'].append(target_category_id)
            games['target_supercategory'].append(target_supercategory)
            games['target_supercategory_id'].append(target_supercategory_id)
            games['categories'] = categories


In [ ]:
%%time
df = pd.DataFrame.from_dict(games)
df = df[['game_id', 'question', 'question_id', 'answer', 'target_category', 
        'target_category_id', 'target_supercategory',
        'target_supercategory_id', 'categories']]
# df.columns